In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# WritingProject

In [35]:
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from typing import Any, Dict, Iterator, List, Optional, Union
import datetime
import random

global_id = 0

def create_content_id():
    global global_id
    now = datetime.datetime.now()
    random_digits = random.randint(100, 999)  # 生成一个四位的随机数

    # 更新 global_id
    global_id += 1
    if global_id > 99999:
        global_id = 0
    global_id_str = str(global_id).zfill(5)

    return now.strftime(f"%Y%m%d.%H%M%S.{global_id_str}.{random_digits}")
    
class BaseContent(BaseModel):
    id: Optional[str] = None
    topic: Optional[str] = None
    path: Optional[str] = None

    def load(self):
        pass  # 实现加载逻辑

    def save(self):
        pass  # 实现保存逻辑

    @root_validator
    def auto_generate_id(cls, value):
        if value['id'] is None:
            value['id'] = create_content_id()   
        return value

class DetailContent(BaseContent):
    text: Optional[str] = None

class OutlineContent(BaseContent):
    children: List[BaseContent] = []

    def add_content(self, content: BaseContent):
        if self.children is None:
            self.children = []
        self.children.append(content)

        return content.id

    def get_content(self, content_id: str) -> Optional[BaseContent]:
        """递归查询并返回指定id的Content"""
        if self.id == content_id:
            return self

        for child in self.children or []:
            if child.id == content_id:
                return child
            if isinstance(child, OutlineContent):
                found = child.get_content(content_id)
                if found:
                    return found

        return None

class Article(OutlineContent):
    task_content_id: Optional[str] = Field(None, description="The content ID of the task")

In [36]:
root = OutlineContent()

t1 = root.add_content(OutlineContent(topic="H1"))
t2 = root.add_content(OutlineContent(topic="H2"))
# t1.add_content(DetailContent(text="my headline 1"))
# t2.add_content(DetailContent(text="my headline 2"))

In [37]:
h1 = root.get_content(t1)
h2_id = h1.add_content(DetailContent(text="my headline 1"))
root.get_content(h2_id)

DetailContent(id='20240504.162201.00004.202', topic=None, path=None, text='my headline 1')

In [40]:
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_zhipu import ChatZhipuAI
from langchain.prompts import ChatPromptTemplate
import json

def get_input(prompt: str = "👤: ") -> str:
    return input(prompt)

class WritingProject(BaseModel):
    """写作任务"""
    
    task: str = None
    max_topics = 10
    max_words = 1024

    _should_continue = True

    _chain = None

    # 任务类型可以包括：
    # - root     最初的写作规划
    # - outlines 写作大纲
    # - detail   细节内容
    task_type = ""

    def next_step(self):
        pass

    def ask_user(self, ai_said: str = ""):
        resp = self.get_input()
        if(resp == "quit"):
            command = "quit"
        elif(resp == "ok"):
            content = ai_said
            command = "ok"
        else:
            content = resp
            command = "chat"

        return content, command

    def get_chain(self):
        prompt = ChatPromptTemplate.from_messages([
            ("system", "你是一个强力助手。"),
            ("assistant", "我是一名AI助手，请向我提问。"),
            ("user", "{question}")
        ])
        llm = ChatZhipuAI()
        return (prompt | llm)

    def run(self):
        # 初始化链
        chain = self.get_chain()

        #
        ai_said = ""

        while(self._should_continue):
            # 用户输入
            resp, command = ask_user(ai_said)
            if command == "quit":
                break
            elif command == "ok":
                # 生成目录或文件

                # 评估任务
                # 如果所有任务都OK，就结束
                
                # 任务分派
                # - 转移到新的子主题
                # - 重新分配session_id
                chain = self.get_chain()
            else:
                # AI推理
                message = chain.invoke(resp)
                if len(message.content) > 0:
                    ai_said = message.content
                